# Resource Quantification Estimation
Here is a simple method for resource quantification using numerical simulation. It leverages the fact that the only way for mRNA or protein to "disappear" from a system is via degradation/dilution.

## Energetic Costs of Individual mRNAs and Proteins
The cost of transcription and translation can be estimated by the amount of phosphate bonds involved in the biosynthesis of each nucleotide or amino acid used as well as for the elongation process. These were determined in experiments by Lynch and Marinov in their paper, [Bioenergetic Costs of a Gene](https://doi.org/10.1073/pnas.1514974112).

### Transcription
An approximate average for the synthesis of nucleoside triphosphates (NTPs) is 46 phosphate bonds, with an additional 2 for polymersiation for a total of 48 phosphate bonds. This is summarised in the following equation, with $\epsilon$ representing the length of the gene sequence in nucleotides.

\begin{align*}
    \text{cost}_{m_i} = \epsilon \cdot 48
\end{align*}

### Translation
The costs for translation are more complicated with each amino acid having varying biosynthesis costs. The approach is to multiply the number of each amino acid in a protein sequence by its associated cost and sum over all amino acids. 

\begin{align*}
    \text{cost}_{x_i} = \sum_{AA} \text{number of AA} \cdot \text{cost of AA}
\end{align*}

The phosphate costs of each amino acid in the table below:
| **Amino Acid** | **One-letter symbol** | **~P Cost** |
|----------------|-----------------------|-------------|
| Ala            | A                     | 11.7        |
| Cys            | C                     | 24.7        |
| Asp            | D                     | 12.7        |
| Glu            | E                     | 15.3        |
| Phe            | F                     | 52.0        |
| Gly            | G                     | 11.7        |
| His            | H                     | 38.3        |
| Ile            | I                     | 32.3        |
| Lys            | K                     | 30.3        |
| Leu            | L                     | 27.3        |
| Met            | M                     | 34.3        |
| Asn            | N                     | 14.7        |
| Pro            | P                     | 20.3        |
| Gln            | Q                     | 16.3        |
| Arg            | R                     | 27.3        |
| Ser            | S                     | 11.7        |
| Thr            | T                     | 18.7        |
| Val            | V                     | 23.3        |
| Trp            | W                     | 74.3        |
| Tyr            | Y                     | 50.0        |

- The total cost of expressing a gene over the life of an organism can be estimated as the individual cost multiplied by the concentration determined from a numerical simulation. 
- We discretise the ODEs and change the differential of a species into difference equations. 
- To calculate the total amount of a species synthesis, we utilise the fact that degradation/dilution terms are linear and that the concentration of a species is calculated at every time point. 
- In this manner, we can simply calculate the amount of a species that has been degraded/diluted (in terms of concentration) to the concentration of the species at steady-state added to the initial concentration to obtain the total amount synthesised (in terms of concentration). 
- Finally, we can multiply the concentration by the volume to obtain the amount. In the case of \emph{E. coli}, the volume can be regarded at $1 \mu M$. 

After numerical simulation, we can extract the time-series data of a species, $A_i$, from the solution vector. This is a sequence of numbers, \{$\mathbf{A}_i$\}. 
\begin{align*}
    \mathbf{A}_i &= \begin{bmatrix}
        A_i [0] \\
        A_i [1] \\
        \vdots \\
        A_i [N] \\
      \end{bmatrix}
\end{align*}

Where $N$ is the number of time points simulated.

The calculation the total number of an mRNA ($m_i$) or protein ($x^\omega$) species simulated up to time, $t$, is as follows:
\begin{align*}
    \dot{m}_i [t] &= m_i [t] - m_i [t-1] = k_{\text{eff}, 1; i} \cdot g^T_i - \delta_i \cdot m_i [t] \\
    m_i [t] &= \sum_1^{\tau = t} (k_{\text{eff}, 1; i} \cdot g^T_i - \delta_i \cdot m_i [\tau]) + m_i[0]\\
    m_{i,\text{ synthesised}} [t] &=  k_{\text{eff}, 1; i} \cdot g^T_i \cdot t + m_i[0] \\
    &= \sum_1^{\tau = t} ( k_{\text{eff}, 1; i} \cdot g^T_i - \delta_i \cdot m_i [\tau] ) + \sum_1^{\tau = t} \delta_i \cdot m_i [\tau] + m_i[0] \\
    &= m_i [t] + m_i[0] + \delta_i \cdot \sum_1^{\tau = t} m_i [\tau]\\
    x^{\omega}_{i,\text{ synthesised}} [t] &= x^{\omega}_i [t] + x^{\omega}_i [0] + \lambda \cdot \sum_1^{\tau = t} x^{\omega}_i [\tau]
\end{align*}

Species are denoted as discrete functions of time to reflect the numerical simulation. This is signified by the square brackets around the variable, $t$. The amount synthesised over the number of time points simulated, $N$, would be $m_{i, \text{ synthesised}} [N]$ and $x^{\omega}_{i,\text{ synthesised}} [N]$. When additional reactions that involve a species are present, we can simply perform a linear addition of the synthesis terms of the additional related species.
